In [118]:
import logging
import pandas as pd
import azure.functions as func
import logging
from azure.storage.blob import BlobServiceClient
import os
import pyodbc
import re

In [158]:
incoming_input_pdf_nbd = "pdf/processed/Account_Details_03Jun2023_133827_NBD_2023-06-14T17:05:30.3232782Z.pdf"
incoming_input_pdf_cbd = "pdf/processed/Account_Details_03Jun2023_133827_CBD_2023-06-14T17:05:30.3232782Z.pdf"
incoming_input_pdf_cltrak = "pdf/processed/Account_Details_03Jun2023_133827_CLTRAK_2023-06-14T17:05:30.3232782Z.pdf"

In [159]:
template_file_path = "https://arunakcs.blob.core.windows.net/excelfiles/main_template/test_template.xlsx"
df_template = pd.read_excel(template_file_path)
df_reference = pd.read_excel(template_file_path, sheet_name="term_references")
server = "akcserver.database.windows.net"
database = "dbarunsql"
username = "Arun"
password = "Asds@2022"
table_name = "TransactionDetails"

In [160]:
try:
    # Establish the database connection
    conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    conn = pyodbc.connect(conn_str)

    # Read rows from the SQL table
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Get column names from the cursor description
    column_names = [column[0] for column in cursor.description]

    # Convert rows to a list of dictionaries
    data = []
    for row in rows:
        data.append(dict(zip(column_names, row)))

    # Create DataFrame from the data
    df_input_bank_statement_from_sql = pd.DataFrame(data)

except Exception as e:
    raise

finally:
    conn.close()

In [161]:
df_input_bank_statement_from_sql.head()

,TransactionId,TransactionDate,Narration,Debit,Credit,RunningBalance,ModelCopyDateTime,file_name,DomainName,CreatedDate,ModifiedDate,Classified,LabelTerm
0,6120,2023-02-06,SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERN...,0.0,5394.94,188329.87,2023-06-14 20:54:36.053,Account_Details_03Jun2023_133827.pdf,NBD,2023-06-14 20:54:36.053,2023-06-14 20:54:36.053,No,None
1,6121,2023-02-06,SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERN...,0.0,5394.94,188329.87,2023-06-14 20:54:36.360,Account_Details_03Jun2023_133827.pdf,NBD,2023-06-14 20:54:36.360,2023-06-14 20:54:36.360,No,None
2,6122,2023-02-06,SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERN...,0.0,5394.94,188329.87,2023-06-14 20:54:36.423,Account_Details_03Jun2023_133827.pdf,NBD,2023-06-14 20:54:36.423,2023-06-14 20:54:36.423,No,None
3,6123,2023-02-06,SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERN...,0.0,5394.94,188329.87,2023-06-14 20:54:36.373,Account_Details_03Jun2023_133827.pdf,NBD,2023-06-14 20:54:36.373,2023-06-14 20:54:36.373,No,None
4,6124,2023-02-06,SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERN...,0.0,5394.94,188329.87,2023-06-14 20:54:36.680,Account_Details_03Jun2023_133827.pdf,NBD,2023-06-14 20:54:36.680,2023-06-14 20:54:36.680,No,None


In [162]:
from datetime import datetime

In [163]:
def read_user_input_data(input_file, df_input):
    current_datetime = datetime.now()
    Current_Date= current_datetime.strftime("%Y-%m-%d")
    filter_date = pd.to_datetime('2023-06-14').date()
    print(filter_date)
    df_input['ModelDate'] = pd.to_datetime(df_input['ModelCopyDateTime'])
    df_input['ModelDateExtracted']=df_input['ModelDate'].dt.date
    df_input =  df_input[df_input['ModelDateExtracted']==filter_date]
    if 'NBD' in input_file:
        nbd_df = df_input[df_input['DomainName']=='NBD']  
        return nbd_df
    
    elif 'CLTRAK' in input_file:
        clt_rak_df = df_input[df_input['DomainName']=='CLTRAK']
        return clt_rak_df
    
    elif 'CBD' in input_file:
        cbd_df = df_input[df_input['DomainName']=='CBD']
        return cbd_df
    
    else: 
        print(f'{input_file} Does not exist.')

In [164]:
out = read_user_input_data(incoming_input_pdf_nbd, df_input_bank_statement_from_sql)

2023-06-14


In [165]:
out['TransactionDate'].isnull().sum()

64

In [166]:
def general_preprocess(input_df):
    dropped_df = input_df['TransactionDate'].dropna()
    return input_df

def preprocess_text_data(input_df):
    lowered_narration_data_series = input_df['Narration'].apply(lambda x: str(x).lower() if x is not None else None)
    input_df['lowered_narration'] = lowered_narration_data_series
    return input_df

In [167]:
a = general_preprocess(out)


In [168]:
new_nbd = preprocess_text_data(a)

In [170]:
def preprocess_template_data(input_dataframe):
    lowered_description = input_dataframe['Description'].apply(str.lower)
    input_dataframe['description_lowered'] = lowered_description
    return input_dataframe

In [171]:
new_template_df = preprocess_template_data(df_template)

In [172]:
df_reference

,Terminology,References
0,Cheques Deposited,.outward t/t.*
1,Inward Remittance,.*inward remittance.*
2,Bounced / Returned cheque/DDR,NaN
3,Cash Deposited,.*sdm deposit.*
4,Internal transfer,inward remittance|inter group
5,Credit card payment,.*credit card.*
6,Cheques issued,NaN
7,Bank Charges,^(?!.*visa charges).*charges.*value added
8,Cash Withdrawn,NaN
9,Card Credits,.*network international.*|.*amex.*|.*fds-tos.*


In [173]:
#new_nbd = new_nbd.dropna(subset=['Narration'])
new_nbd_without_nulls = new_nbd.dropna(subset=['Narration'])


In [174]:
new_transactions = new_nbd_without_nulls['Narration']
print(len(new_transactions))

1168


In [191]:
from joblib import dump, load
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Load the saved model
model = load('AkcsNlpCustommodel.pkl')

# Load the saved vectorizer
vectorizer = load('vectorizer.pkl')

# New data
new_transactions = new_nbd_without_nulls['Narration']


# Vectorize the new data
new_transactions_vectorized = vectorizer.transform(new_transactions)

# Predict using the loaded model
predictions = model.predict(new_transactions_vectorized)

# Print the predictions
for transaction, prediction in zip(new_transactions, predictions):
    print(f'Transaction: {transaction}\nPrediction: {prediction}')

new_nbd['Prediction'] = predictions

Transaction: SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERNATIONAL LLC NI POS SETT 010623 130437015 134926238 - S38917312
Prediction: Card Credits
Transaction: SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERNATIONAL LLC NI POS SETT 010623 130437015 134926238 - S38917312
Prediction: Card Credits
Transaction: SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERNATIONAL LLC NI POS SETT 010623 130437015 134926238 - S38917312
Prediction: Card Credits
Transaction: SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERNATIONAL LLC NI POS SETT 010623 130437015 134926238 - S38917312
Prediction: Card Credits
Transaction: SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERNATIONAL LLC NI POS SETT 010623 130437015 134926238 - S38917312
Prediction: Card Credits
Transaction: SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERNATIONAL LLC NI POS SETT 010623 860024026 134930811 - S38906975
Prediction: Card Credits
Transaction: SB-TRANSFERSB REF. 32070946 B/O NETWORK INTERNATIONAL LLC NI POS SETT 010623 860024026 134930811 - S38906

C:\Users\Admin\AppData\Local\Temp\ipykernel_1588\3008061877.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_nbd['Prediction'] = predictions


In [193]:
new_nbd['Prediction'].value_counts()

Unclassified         528
Cash Deposited       256
Card Credits         192
Bank Charges          64
Inward Remittance     64
Uber/Careem           48
Internal transfer     16
Name: Prediction, dtype: int64

In [195]:
misclassified= new_nbd[new_nbd['Prediction'] == 'Unclassified']

In [199]:
misclassified.loc[1554]['Narration']

'GOUS SAA BEING COMMISION PA COMMISION 25-87040938-1- 151 - AE0334747'

In [203]:
new_nbd[new_nbd['Prediction'] == 'Bank Charges']['Debit'].sum()

12.799999999999999

In [208]:
new_nbd[new_nbd['Prediction'] == 'Cash Deposited']['Credit'].sum()

1339200.0

In [212]:
df_template

,Description,Emirates NBD-Classic Luxury-Main,Rak Bank-Classic Luxury,CLT - ADCB,CBD Bank,EIB - Loan account,OLT - Emirates Islamic Bank,Emirates NBD-Classic Passenger,ENBD - Classic Riders,description_lowered
0,Opening Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,opening balance
1,Cheques Deposited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheques deposited
2,Inward Remittance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inward remittance
3,Bounced / Returned cheque/DDR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bounced / returned cheque/ddr
4,Cash Deposited,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cash deposited
5,Internal transfer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,internal transfer
6,Credit card payment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit card payment
7,Cheques issued,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheques issued
8,Bank Charges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bank charges
9,Cash Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cash withdrawn


In [211]:
df_template.loc[df_template['Description'] == 'Inward Remittance', 'Emirates NBD-Classic Luxury-Main']

2   NaN
Name: Emirates NBD-Classic Luxury-Main, dtype: float64

In [229]:
new_descriptions  = df_template['Description'].unique()

In [241]:
def populate_final_report(report_template, nlp_classified_df, input_file_path):
    if 'NBD' in input_file_path:
        
        for description in report_template['Description']:
            filtered_df = nlp_classified_df[nlp_classified_df['Prediction'] == description]
            debit_sum = filtered_df['Debit'].sum()
            credit_sum = filtered_df['Credit'].sum()
            total_sum = credit_sum + (-debit_sum)
            report_template.loc[report_template['Description'] == description, 'Emirates NBD-Classic Luxury-Main'] = total_sum
        report_template.loc[report_template['Description'] == 'Closing Balance at the day end', 'Emirates NBD-Classic Luxury-Main'] = report_template['Emirates NBD-Classic Luxury-Main'][1:12].sum()
        
        return report_template

In [242]:
populate_final_report(df_template, new_nbd, incoming_input_pdf_nbd)

,Description,Emirates NBD-Classic Luxury-Main,Rak Bank-Classic Luxury,CLT - ADCB,CBD Bank,EIB - Loan account,OLT - Emirates Islamic Bank,Emirates NBD-Classic Passenger,ENBD - Classic Riders,description_lowered
0,Opening Balance,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,opening balance
1,Cheques Deposited,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheques deposited
2,Inward Remittance,439328.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inward remittance
3,Bounced / Returned cheque/DDR,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bounced / returned cheque/ddr
4,Cash Deposited,1339200.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cash deposited
5,Internal transfer,800000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,internal transfer
6,Credit card payment,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit card payment
7,Cheques issued,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheques issued
8,Bank Charges,-12.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bank charges
9,Cash Withdrawn,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cash withdrawn


In [235]:

# # Iterate over each description in the new DataFrame
# for description in df_template['Description']:
#     # Filter previous DataFrame based on matching prediction
#     filtered_df = new_nbd[new_nbd['Prediction'] == description]
    
#     # Calculate the sum of debit and credit values
#     debit_sum = filtered_df['Debit'].sum()
#     credit_sum = filtered_df['Credit'].sum()
#     total_sum = credit_sum + (-debit_sum)
#     df_template.loc[df_template['Description'] == description, 'Emirates NBD-Classic Luxury-Main'] = total_sum



In [237]:
df_template.loc[df_template['Description'] == 'Closing Balance at the day end', 'Emirates NBD-Classic Luxury-Main'] = df_template['Emirates NBD-Classic Luxury-Main'][1:12].sum()

In [238]:
df_template

,Description,Emirates NBD-Classic Luxury-Main,Rak Bank-Classic Luxury,CLT - ADCB,CBD Bank,EIB - Loan account,OLT - Emirates Islamic Bank,Emirates NBD-Classic Passenger,ENBD - Classic Riders,description_lowered
0,Opening Balance,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,opening balance
1,Cheques Deposited,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheques deposited
2,Inward Remittance,439328.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inward remittance
3,Bounced / Returned cheque/DDR,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bounced / returned cheque/ddr
4,Cash Deposited,1339200.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cash deposited
5,Internal transfer,800000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,internal transfer
6,Credit card payment,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,credit card payment
7,Cheques issued,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cheques issued
8,Bank Charges,-12.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bank charges
9,Cash Withdrawn,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cash withdrawn
